In [17]:
!python -m serial.tools.list_ports

COM96               
COM106              
COM107              


3 ports found


In [38]:
serAT.close()
serTIM.close()

NameError: name 'serTIM' is not defined

In [25]:
import serial
# serTIM = serial.Serial('COM121', 115200, timeout=1)
serAT = serial.Serial('COM106', 9600, timeout=1)
# serGPS = serial.Serial('COM83', 9600, timeout=3)

In [23]:
# from process.globals import *

In [26]:
import colorama
import sys
import time
import pytest
import numpy as np
import math
import time
import pynmea2

from process import timing_bits as p
from process import find as find
# from process import serialcomms as s

In [27]:
yellow = ''
cyan = ''
magenta = ''

def sendTIM(cmd):
    print(yellow + cmd)
    serTIM.write(bytes(cmd + '\r', 'utf-8'))

def receiveTIM():
    c = 0
    a = []
    b = []
    g = 0
    now = time.time()
    while True and (time.time() - now < 600.0):
        d = serTIM.readline().decode('utf-8')
        if len(d):
            d = d.strip()
            a.append(d)
            b.append(int(d.split(',')[0]))
            print(white + d, blue + str(b))
            g = find.guess_seq_len(b)
            if g != 0:
                print(magenta + str(g))
                break
    
def sendAT(cmd, t=0, expect='OK'):
    print(yellow + cmd)
    serAT.write(bytes(cmd + '\r', 'utf-8'))
    return receiveAT(t, expect)

def receiveAT(t=0, expect='OK'):
    c = 0
    data = []
    while True:
        d = serAT.readline().decode('utf-8')
        if not len(d):
            c += 1
        d = d.strip()
        if len(d) > 0:
            print(cyan + d)
            out = p.converter(d)
            if out:
                print(magenta + out)
            data.append(d)
        if t > 0:
            if c == t:
                data.append('timeout')
                return data
        if expect in d or 'ERROR' in d:
            return data

In [28]:
to_bin = lambda x, n: format(x, 'b').zfill(n)

In [29]:
nw_edrx = ["0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111","0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111","0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111","0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111","0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111","0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111","0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111","0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111","0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111","0010","0011","0010","0101","0010","1001","1010","1011","1100","1101","1110","1111"]
nw_ptw  = ["0000","0000","0000","0000","0000","0000","0000","0000","0000","0000","0000","0000","0001","0001","0001","0001","0001","0001","0001","0001","0001","0001","0001","0001","0010","0010","0010","0010","0010","0010","0010","0010","0010","0010","0010","0010","0011","0011","0011","0011","0011","0011","0011","0011","0011","0011","0011","0011","0100","0100","0100","0100","0100","0100","0100","0100","0100","0100","0100","0100","0101","0101","0101","0101","0101","0101","0101","0101","0101","0101","0101","0101","0110","0110","0110","0110","0110","0110","0110","0110","0110","0110","0110","0110","0111","0111","0111","0111","0111","0111","0111","0111","0111","0111","0111","0111","1000","1000","1000","1000","1000","1000","1000","1000","1000","1000","1000","1000","1001","1001","1001","1001","1001","1001","1001","1001","1001","1001","1001","1001"]

In [30]:
def OK(cmd, t=0):
    assert 'OK' in sendAT(cmd, t)

def expect(cmd, reply, t=1):
    data = sendAT(cmd, t, reply)
    assert True in [reply in i for i in data]
    return data

In [31]:
def serialConnected():
    assert serAT.is_open == True
    assert serTIM.is_open == True
    OK('AT')

In [32]:
def initURC():
    OK('AT+NCONFIG="AUTOCONNECT","FALSE"')
    OK('AT+NCONFIG="CR_0859_SI_AVOID","TRUE"')
    OK('AT+NCONFIG="CR_0354_0338_SCRAMBLING","TRUE"')
    OK('AT+CMEE=1')
    OK('AT+CSCON=1')
    OK('AT+CEREG=5')
    OK('AT+NPSMR=1')

 Set modem to full functionality.

In [33]:
def fullFuctionality():
    OK('AT+CFUN=0', 3)
    # expect('at+cfun?', '+CFUN: 0', 1)
    expect('AT+CFUN=1', '+CEREG: 0', 2)
    receiveAT()

Network registration attempt

In [34]:
def registerNetwork():
    expect('AT+COPS=0', '+CEREG: 1', 10)

Network registration successful

In [35]:
def successfulRegistration():
    expect('AT+CEREG?', '+CEREG: 5,1')

Ping test

In [36]:
def ping():
    expect('at+nping="8.8.8.8"', '+NPING: "8.8.8.8"', 10)

Release from network

In [37]:
def releaseNetwork():
    expect('at+cops=2', '+NPSMR: 1', 10)
    expect('at+cops=0', '+CEREG: 1', 10)
    OK('AT+NSOCR="DGRAM",17,14000,1')
    expect('AT+NSOSTF=0,"1.1.1.1",7,0x200,1,"FF"', '+CSCON: 0', 10)
    OK('at+nsocl=0')

Setup

In [20]:
def setup():
    serialConnected()
    initURC()
    fullFuctionality()
    registerNetwork()
    successfulRegistration()
    ping()
#     releaseNetwork()

In [21]:
setup()

AT
OK
AT+NCONFIG="AUTOCONNECT","FALSE"
OK
AT+NCONFIG="CR_0859_SI_AVOID","TRUE"
OK
AT+NCONFIG="CR_0354_0338_SCRAMBLING","TRUE"
OK
AT+CMEE=1
OK
AT+CSCON=1
OK
AT+CEREG=5
OK
AT+NPSMR=1
OK
AT+CFUN=0
OK
AT+CFUN=1
+CEREG: 0,"0000","0",7,,,,
OK
AT+COPS=0
OK
+CEREG: 2,"0000","0",7,,,,
+CSCON: 1
+CEREG: 1,"8CA7","28C465",7,,,"00010000",
Active T3324: 32 seconds, Periodic T3412: failed
AT+CEREG?
+CEREG: 5,1,"8CA7","28C465",7,,,"00010000",
at+nping="8.8.8.8"
OK
OK
+NPING: "8.8.8.8",53,518


In [7]:
!python -m serial.tools.list_ports

COM38               
COM39               
COM121              


3 ports found


In [8]:
!pytest -vk connect

============================= test session starts =============================
platform win32 -- Python 3.7.3, pytest-4.6.2, py-1.8.0, pluggy-0.12.0 -- c:\users\d7rob\appdata\local\programs\python\python37\python.exe
cachedir: .pytest_cache
rootdir: C:\GIT\masters\code\tests, inifile: pytest.ini
collecting ... collected 17 items / 16 deselected / 1 selected

test_eDRX.py::test_connect PASSED                                        [100%]

=================== 1 passed, 16 deselected in 0.29 seconds ===================


In [9]:
tIDRX = [1.28, 2.56, 5.12, 7.68]
tEDRX = [10.24, 20.48, 40.96, 81.92, 163.84,
         327.68, 655.36, 1310.72, 1966.08, 2621.44]
tPTW = [2.56, 5.12, 7.68, 10.24, 12.8, 15.36, 17.92, 20.48,
        23.04, 25.6, 28.16, 30.72, 33.28, 35.84, 38.4, 40.96]
tActive = [2, 4, 6, 8, 10, 12, 14, 60, 120, 180, 240,
           300, 360, 420, 720, 1080, 1440, 1800, 2160, 2520]
tPeriodicTau = [2, 4, 6, 8, 10, 12, 14, 30, 60, 90, 120, 150, 180, 210, 240, 300, 360, 420, 600, 1200, 1800, 2400, 3000, 3600, 4200, 7200, 10800,
                14400, 18000, 21600, 25200, 36000, 72000, 108000, 144000, 180000, 216000, 252000, 1152000, 2304000, 3456000, 4608000, 5760000, 6912000, 8064000]

In [14]:
print(len(tEDRX), len(tPTW), len(tActive), len(tPeriodicTau))

10 16 20 45


In [15]:
10 * 16 * 20 * 45

144000

In [67]:
r = ['NUESTATS: "RADIO","Signal power",-487', 'NUESTATS: "RADIO","Total power",-401', 'NUESTATS: "RADIO","TX power",-70', 'NUESTATS: "RADIO","TX time",2017', 'NUESTATS: "RADIO","RX time",10709', 'NUESTATS: "RADIO","Cell ID",2671717', 'NUESTATS: "RADIO","ECL",0', 'NUESTATS: "RADIO","SNR",110', 'NUESTATS: "RADIO","EARFCN",3712', 'NUESTATS: "RADIO","PCI",234', 'NUESTATS: "RADIO","RSRQ",-110', 'NUESTATS: "BLER","RLC UL BLER",0', 'NUESTATS: "BLER","RLC DL BLER",0', 'NUESTATS: "BLER","MAC UL BLER",0', 'NUESTATS: "BLER","MAC DL BLER",0', 'NUESTATS: "BLER","Total TX bytes",1952', 'NUESTATS: "BLER","Total RX bytes",1482', 'NUESTATS: "BLER","Total TX blocks",51', 'NUESTATS: "BLER","Total RX blocks",51', 'NUESTATS: "BLER","Total RTX blocks",0', 'NUESTATS: "BLER","Total ACK/NACK RX",0', 'NUESTATS: "THP","RLC UL",0', 'NUESTATS: "THP","RLC DL",0', 'NUESTATS: "THP","MAC UL",0', 'NUESTATS: "THP","MAC DL",0', 'NUESTATS: "APPSMEM","Current Allocated":16096', 'NUESTATS: "APPSMEM","Total Free":6904', 'NUESTATS: "APPSMEM","Max Free":6904', 'NUESTATS: "APPSMEM","Num Allocs":1001', 'NUESTATS: "APPSMEM","Num Frees":937', 'OK']
r = r[:-1]
r

['NUESTATS: "RADIO","Signal power",-487',
 'NUESTATS: "RADIO","Total power",-401',
 'NUESTATS: "RADIO","TX power",-70',
 'NUESTATS: "RADIO","TX time",2017',
 'NUESTATS: "RADIO","RX time",10709',
 'NUESTATS: "RADIO","Cell ID",2671717',
 'NUESTATS: "RADIO","ECL",0',
 'NUESTATS: "RADIO","SNR",110',
 'NUESTATS: "RADIO","EARFCN",3712',
 'NUESTATS: "RADIO","PCI",234',
 'NUESTATS: "RADIO","RSRQ",-110',
 'NUESTATS: "BLER","RLC UL BLER",0',
 'NUESTATS: "BLER","RLC DL BLER",0',
 'NUESTATS: "BLER","MAC UL BLER",0',
 'NUESTATS: "BLER","MAC DL BLER",0',
 'NUESTATS: "BLER","Total TX bytes",1952',
 'NUESTATS: "BLER","Total RX bytes",1482',
 'NUESTATS: "BLER","Total TX blocks",51',
 'NUESTATS: "BLER","Total RX blocks",51',
 'NUESTATS: "BLER","Total RTX blocks",0',
 'NUESTATS: "BLER","Total ACK/NACK RX",0',
 'NUESTATS: "THP","RLC UL",0',
 'NUESTATS: "THP","RLC DL",0',
 'NUESTATS: "THP","MAC UL",0',
 'NUESTATS: "THP","MAC DL",0',
 'NUESTATS: "APPSMEM","Current Allocated":16096',
 'NUESTATS: "APPSMEM","T

In [82]:
dr = {}
for i in r:
    j = i.split(',')
    if j[0] != 'NUESTATS: "APPSMEM"':
        dr[j[1]] = j[2]
    else:
        dr[j[1].split(':')[0]] = j[1].split(':')[1]
    
dr
for key in dr:
    print(dr[key], end=",")

-487,-401,-70,2017,10709,2671717,0,110,3712,234,-110,0,0,0,0,1952,1482,51,51,0,0,0,0,0,0,16096,6904,6904,1001,937,